<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Random_Forest_and_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Load the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/contracts_with_new_data_MEAN.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

<ipython-input-4-283dbef524b6>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


   Unnamed: 0    ANO_SID CORPORATE_DEVISION  ORTPLZ  ORTS-NAME  \
0           0  4114028.0                VHV   42109  Wuppertal   
1           1  4114039.0                VHV   42277  Wuppertal   
2           2  4114045.0                VHV   42389  Wuppertal   
3           3  4114049.0                VGV   42277  Wuppertal   
4           4  4114055.0                VHV   42553    Velbert   

                STRASSE  SUM_INSURED CONSTRACTION_DESIGN  CONSTRUCTION_YEAR  \
0     Hans-Böckler-Str.      71081.0      NORMAL_VENTURE        1967.565648   
1       Liegnitzer Str.      55708.0      NORMAL_VENTURE        1967.565648   
2             Rascheweg      74148.0      DESIGN_CLASS_I        1967.565648   
3               Am Diek     664000.0      NORMAL_VENTURE        1967.565648   
4  Emil-Schniewind-Str.      75682.0      NORMAL_VENTURE        1967.565648   

         WFL  ... PRIOR_DAMAGES  UVV-KZ  UNDERWRITER              PARTY-ID  \
0   69.00000  ...             0       1           

In [5]:
print(contract_class['DAMAGE'].value_counts())


na_counts = contract_class.isna().sum()

print(na_counts)

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
Unnamed: 0                      0
ANO_SID                        50
CORPORATE_DEVISION              0
ORTPLZ                          0
ORTS-NAME                      15
STRASSE                      8516
SUM_INSURED                 14832
CONSTRACTION_DESIGN       1083360
CONSTRUCTION_YEAR               0
WFL                         14996
ZONE                       877533
SF-SYSTEM                 8713940
TYPE_OF_DEDUCTIBLE              0
DRAIN_PIPE_INSURED              0
PRODUCTLINE               1583747
PRIOR_DAMAGES                   0
UVV-KZ                          0
UNDERWRITER                     0
PARTY-ID                      191
contract_year                   0
PIPE_PREMIUM_AMOUNT           164
YEAR                            0
DAMAGE                          0
DAMAGE_FLOOD_ZONE               0
DAMAGE_HEAVY_RAIN_ZONE          0
dtype: int64


# Models

## Preprocess the data

In [6]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['ANO_SID','CORPORATE_DEVISION','ORTPLZ','ZONE', 'PRODUCTLINE', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE']]

print(initial_features['DAMAGE'].value_counts())

initial_features_clean = initial_features.dropna()

print(initial_features_clean['DAMAGE'].value_counts())

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
DAMAGE
0    7680154
1     112491
Name: count, dtype: int64


In [7]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

     ANO_SID  ORTPLZ  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  \
0  4114028.0   42109        1967.565648   69.0                   0   
1  4114039.0   42277        1967.565648   65.0                   0   
2  4114045.0   42389        1967.565648   75.0                   0   
4  4114055.0   42553        1967.565648  119.0                   0   
5  4114057.0   42113        1967.565648  100.0                   0   

   PRIOR_DAMAGES  UVV-KZ  DAMAGE  DAMAGE_FLOOD_ZONE  DAMAGE_HEAVY_RAIN_ZONE  \
0              0       1       0                1.0                1.727273   
1              0       1       0                1.0                1.931034   
2              0       1       0                1.0                1.962963   
4              0       1       0                1.0                1.768000   
5              0       1       0                1.0                1.824324   

   ...  ZONE_4  ZONE_4.0  ZONE_5  ZONE_5.0  ZONE_6  ZONE_6.0  ZONE_7  \
0  ...   False     False   False

## Random Forest

In [8]:
# Define feature and target

X = features_binary.drop(columns=['ANO_SID','DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

In [9]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)


In [10]:
# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))
# With undersampling accuracy gets worse, but we detect more TRUE Positives. Still way more false positive

Accuracy: 0.7693870309760037
Confusion Matrix:
[[1182464  353524]
 [   5893   16648]]
True Positives: 16648
False Positives: 353524
True Negatives: 1182464
False Negatives: 5893
              precision    recall  f1-score   support

           0       1.00      0.77      0.87   1535988
           1       0.04      0.74      0.08     22541

    accuracy                           0.77   1558529
   macro avg       0.52      0.75      0.48   1558529
weighted avg       0.98      0.77      0.86   1558529



In [11]:
# Feature-importnace analysis
feature_importance = rf_classifier.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
ORTPLZ: 0.19533521377215407
CONSTRUCTION_YEAR: 0.14470950311335481
WFL: 0.17196384642395823
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.005339127571436116
UVV-KZ: 0.013669806617498146
DAMAGE_FLOOD_ZONE: 0.06159186743348311
DAMAGE_HEAVY_RAIN_ZONE: 0.15287015508671584
CORPORATE_DEVISION_VHV: 0.15697117376544298
CONSTRACTION_DESIGN_CARAVAN_MOTORHOME: 7.219867942986655e-06
CONSTRACTION_DESIGN_DESIGN_CLASS_I: 0.00025946753653168924
CONSTRACTION_DESIGN_DESIGN_CLASS_II: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_III: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_IV: 0.0
CONSTRACTION_DESIGN_DESIGN_CLASS_V: 3.1502769315460096e-07
CONSTRACTION_DESIGN_NORMAL_VENTURE: 0.001125648718421881
CONSTRACTION_DESIGN_PREDOMINANTLY_WOODEN_ROOF: 0.00016591472812097174
CONSTRACTION_DESIGN_PREFAB_HOUSE: 0.000977190087321449
CONSTRACTION_DESIGN_PREFAB_HOUSE_I: 0.0
PRODUCTLINE_Kompakt: 0.012119545985870091
PRODUCTLINE_Plus: 0.002588631439404429
PRODUCTLINE_Premium: 0.0021154620100385557
PRODUCTLINE_Sonst: 0.02

## Random Forest with SMOOTE Resampling

In [36]:
# Define feature and target
X = features_binary.drop(columns=['ANO_SID','DAMAGE'])
y = features_binary['DAMAGE']

# Split into test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Resample the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Print the shape of resampled data
print('Original dataset shape:', y_train.value_counts())
print('Resampled dataset shape:', y_resampled.value_counts())

Original dataset shape: DAMAGE
0    6144166
1      89950
Name: count, dtype: int64
Resampled dataset shape: DAMAGE
0    6144166
1    6144166
Name: count, dtype: int64


In [16]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)

In [17]:
# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.9743245072757709
Confusion Matrix:
[[1510358   25630]
 [  14386    8155]]
True Positives: 8155
False Positives: 25630
True Negatives: 1510358
False Negatives: 14386
              precision    recall  f1-score   support

           0       0.99      0.98      0.99   1535988
           1       0.24      0.36      0.29     22541

    accuracy                           0.97   1558529
   macro avg       0.62      0.67      0.64   1558529
weighted avg       0.98      0.97      0.98   1558529



## Random Forest with Rolling window

In [31]:
#Choose variables where it makes logical sense

initial_features = contract_class.loc[:, ['ANO_SID','CORPORATE_DEVISION','ORTPLZ','ZONE', 'PRODUCTLINE', 'CONSTRACTION_DESIGN','CONSTRUCTION_YEAR','WFL','ZONE','DRAIN_PIPE_INSURED', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER','DAMAGE','DAMAGE_FLOOD_ZONE','DAMAGE_HEAVY_RAIN_ZONE','YEAR']]

print(initial_features['DAMAGE'].value_counts())

initial_features_clean = initial_features.dropna()

print(initial_features_clean['DAMAGE'].value_counts())

DAMAGE
0    10203688
1      170852
Name: count, dtype: int64
DAMAGE
0    7680154
1     112491
Name: count, dtype: int64


In [23]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)


In [24]:
# Define feature and target

X = features_binary.drop(columns=['DAMAGE', 'ANO_SID'])
y = features_binary['DAMAGE']



In [28]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = X_test.copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('combined_probabilities_all_years.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())



  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    692820
           1       0.33      0.27      0.29      9020

    accuracy                           0.98    701840
   macro avg       0.66      0.63      0.64    701840
weighted avg       0.98      0.98      0.98    701840

Confusion Matrix for year 2015:
[[687958   4862]
 [  6621   2399]]
True Positives: 2399
False Positives: 4862
True Negatives: 687958
False Negatives: 6621


 10%|█         | 1/10 [00:30<04:31, 30.13s/it]

Classification report for year 2016:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    719837
           1       0.38      0.27      0.31     10068

    accuracy                           0.98    729905
   macro avg       0.68      0.63      0.65    729905
weighted avg       0.98      0.98      0.98    729905

Confusion Matrix for year 2016:
[[715444   4393]
 [  7396   2672]]
True Positives: 2672
False Positives: 4393
True Negatives: 715444
False Negatives: 7396


 20%|██        | 2/10 [01:02<04:12, 31.60s/it]

...

Classification report for year 2023:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    716445
           1       0.18      0.15      0.16     10366

    accuracy                           0.98    726811
   macro avg       0.59      0.57      0.58    726811
weighted avg       0.98      0.98      0.98    726811

Confusion Matrix for year 2023:
[[709686   6759]
 [  8854   1512]]
True Positives: 1512
False Positives: 6759
True Negatives: 709686
False Negatives: 8854


 90%|█████████ | 9/10 [04:57<00:33, 33.84s/it]

Classification report for year 2024:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99    721440
           1       0.04      0.14      0.06      1900

    accuracy                           0.99    723340
   macro avg       0.52      0.56      0.53    723340
weighted avg       1.00      0.99      0.99    723340

Confusion Matrix for year 2024:
[[714719   6721]
 [  1643    257]]
True Positives: 257
False Positives: 6721
True Negatives: 714719
False Negatives: 1643


100%|██████████| 10/10 [05:32<00:00, 33.26s/it]


Year: 2015, Recall Score: 0.2659645232815965
Year: 2016, Recall Score: 0.2653953118792213
Year: 2017, Recall Score: 0.2491823385118561
Year: 2018, Recall Score: 0.16120140671038874
Year: 2019, Recall Score: 0.17048323900740095
Year: 2020, Recall Score: 0.17696397813225492
Year: 2021, Recall Score: 0.1775091021501683
Year: 2022, Recall Score: 0.16787844130381505
Year: 2023, Recall Score: 0.14586147019100906
Year: 2024, Recall Score: 0.13526315789473684
Combined probabilities for all years:
   ORTPLZ  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  PRIOR_DAMAGES  \
0   42109        1967.565648   69.0                   0              0   
1   42277        1967.565648   65.0                   0              0   
2   42389        1967.565648   75.0                   0              0   
3   42553        1967.565648  119.0                   0              0   
4   42113        1967.565648  100.0                   0              0   

   UVV-KZ  DAMAGE_FLOOD_ZONE  DAMAGE_HEAVY_RAIN_ZONE  YEAR  \

In [32]:
# Count the entries in the combined probabilities DataFrame



num_entries = combined_all_years.shape[0]

print("Number of entries in the combined probabilities DataFrame:", num_entries)

Number of entries in the combined probabilities DataFrame: 7119543


## Random Forrest with expanding window

In [35]:

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [10, 20],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'].isin(train_years)]
    y_train = y[features_binary['YEAR'].isin(train_years)]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)

    # Initialize the SMOTE resampler
    smote = SMOTE(random_state=42)

    # Create a pipeline with SMOTE and Random Forest classifier
    pipeline = Pipeline([
        ('smote', smote),
        ('rf', rf_classifier)
    ])

    # Perform Grid Search with cross-validation
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2, scoring='recall')
    grid_search.fit(X_train, y_train)

    # Get the best model from Grid Search
    best_rf_classifier = grid_search.best_estimator_

    # Make predictions (probabilities)
    y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for year {test_year}: {grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters for year 2015: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Classification report for year 2015:


 10%|█         | 1/10 [06:55<1:02:23, 415.99s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    692820
           1       0.79      0.21      0.33      9020

    accuracy                           0.99    701840
   macro avg       0.89      0.60      0.66    701840
weighted avg       0.99      0.99      0.99    701840

Confusion Matrix for year 2015:
[[692323    497]
 [  7139   1881]]
True Positives: 1881
False Positives: 497
True Negatives: 692323
False Negatives: 7139
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters for year 2016: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10}
Classification report for year 2016:


 20%|██        | 2/10 [22:24<1:35:40, 717.55s/it]

...

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    713366
           1       0.33      0.13      0.19     12241

    accuracy                           0.98    725607
   macro avg       0.66      0.56      0.59    725607
weighted avg       0.97      0.98      0.98    725607

Confusion Matrix for year 2022:
[[710071   3295]
 [ 10598   1643]]
True Positives: 1643
False Positives: 3295
True Negatives: 710071
False Negatives: 10598
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters for year 2023: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Classification report for year 2023:


 90%|█████████ | 9/10 [8:01:40<1:22:42, 4962.87s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    716445
           1       0.30      0.12      0.17     10366

    accuracy                           0.98    726811
   macro avg       0.64      0.56      0.58    726811
weighted avg       0.98      0.98      0.98    726811

Confusion Matrix for year 2023:
[[713638   2807]
 [  9161   1205]]
True Positives: 1205
False Positives: 2807
True Negatives: 713638
False Negatives: 9161
Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters for year 2024: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 20}
Classification report for year 2024:


100%|██████████| 10/10 [10:24:51<00:00, 3749.13s/it]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    721440
           1       0.25      0.07      0.12      1900

    accuracy                           1.00    723340
   macro avg       0.62      0.54      0.56    723340
weighted avg       1.00      1.00      1.00    723340

Confusion Matrix for year 2024:
[[721015    425]
 [  1758    142]]
True Positives: 142
False Positives: 425
True Negatives: 721015
False Negatives: 1758
Year: 2015, Recall Score: 0.20853658536585365
Year: 2016, Recall Score: 0.19586809694080254
Year: 2017, Recall Score: 0.22158626328699918
Year: 2018, Recall Score: 0.12926527896587778
Year: 2019, Recall Score: 0.13417501088376144
Year: 2020, Recall Score: 0.14423725005616717
Year: 2021, Recall Score: 0.10730232877653363
Year: 2022, Recall Score: 0.13422106037088474
Year: 2023, Recall Score: 0.11624541771174995
Year: 2024, Recall Score: 0.07473684210526316


NameError: name 'combined_all_years_df' is not defined

# Naive Bayes

In [14]:
# Define initial features
initial_features = contract_class.loc[:, ['CORPORATE_DEVISION', 'SUM_INSURED', 'CONSTRACTION_DESIGN', 'CONSTRUCTION_YEAR', 'WFL', 'ZONE', 'DRAIN_PIPE_INSURED', 'PRODUCTLINE', 'PRIOR_DAMAGES', 'UVV-KZ', 'UNDERWRITER', 'DAMAGE', 'YEAR']]
initial_features_clean = initial_features.dropna()

# Define feature and target
columns_to_encode = ['CORPORATE_DEVISION', 'CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create binary variables using one-hot encoding
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

X = features_binary.drop(columns=['DAMAGE'])
y = features_binary['DAMAGE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separate continuous and discrete features
continuous_features = ['SUM_INSURED', 'CONSTRUCTION_YEAR']
# The discrete features need to be adjusted after one-hot encoding
discrete_features = list(set(X.columns) - set(continuous_features))

In [ ]:
# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'] == train_year]
    y_train = y[features_binary['YEAR'] == train_year]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

# Train GaussianNB on continuous features
    gnb = GaussianNB()
    gnb.fit(X_train[continuous_features], y_train)

# Train MultinomialNB on discrete features
    mnb = MultinomialNB()
    mnb.fit(X_train[discrete_features], y_train)

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)
# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:03<00:27,  3.02s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    775271
           1       0.42      0.12      0.19      9736

    accuracy                           0.99    785007
   macro avg       0.70      0.56      0.59    785007
weighted avg       0.98      0.99      0.98    785007

Confusion Matrix for year 2015:
[[773616   1655]
 [  8536   1200]]
True Positives: 1200
False Positives: 1655
True Negatives: 773616
False Negatives: 8536
Classification report for year 2016:


 20%|██        | 2/10 [00:06<00:26,  3.28s/it]

...

              precision    recall  f1-score   support

           0       0.98      0.98      0.98    797092
           1       0.04      0.05      0.05     13298

    accuracy                           0.96    810390
   macro avg       0.51      0.52      0.51    810390
weighted avg       0.97      0.96      0.97    810390

Confusion Matrix for year 2022:
[[780519  16573]
 [ 12590    708]]
True Positives: 708
False Positives: 16573
True Negatives: 780519
False Negatives: 12590
Classification report for year 2023:


 90%|█████████ | 9/10 [00:27<00:03,  3.11s/it]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98    800711
           1       0.04      0.07      0.05     11149

    accuracy                           0.96    811860
   macro avg       0.51      0.52      0.51    811860
weighted avg       0.97      0.96      0.97    811860

Confusion Matrix for year 2023:
[[779298  21413]
 [ 10356    793]]
True Positives: 793
False Positives: 21413
True Negatives: 779298
False Negatives: 10356
Classification report for year 2024:


100%|██████████| 10/10 [00:30<00:00,  3.08s/it]

              precision    recall  f1-score   support

           0       1.00      0.96      0.98    806672
           1       0.01      0.08      0.01      2049

    accuracy                           0.96    808721
   macro avg       0.50      0.52      0.49    808721
weighted avg       1.00      0.96      0.98    808721

Confusion Matrix for year 2024:
[[775821  30851]
 [  1877    172]]
True Positives: 172
False Positives: 30851
True Negatives: 775821
False Negatives: 1877
Year: 2015, Recall Score: 0.12325390304026294
Year: 2016, Recall Score: 0.10864539990753583
Year: 2017, Recall Score: 0.09599388086815183
Year: 2018, Recall Score: 0.029195804195804195
Year: 2019, Recall Score: 0.030208333333333334
Year: 2020, Recall Score: 0.03311074012242626
Year: 2021, Recall Score: 0.03690551680405834
Year: 2022, Recall Score: 0.0532410888855467
Year: 2023, Recall Score: 0.07112745537716388
Year: 2024, Recall Score: 0.08394338701805759


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

Accuracy: 0.99

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99   1719308
           1       0.98      0.04      0.08     24428

    accuracy                           0.99   1743736
   macro avg       0.98      0.52      0.54   1743736
weighted avg       0.99      0.99      0.98   1743736


Confusion Matrix:
 [[1719284      24]
 [  23369    1059]]
True Positives: 1059
False Positives: 24
True Negatives: 1719284
False Negatives: 23369


## Naive Bayes with expanding window

In [15]:
# Define parameter grids for GaussianNB and MultinomialNB
gnb_param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

mnb_param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0]
}

# Create a list of unique years in the data
years = sorted(features_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    X_train = X[features_binary['YEAR'].isin(train_years)]
    y_train = y[features_binary['YEAR'].isin(train_years)]
    X_test = X[features_binary['YEAR'] == test_year]
    y_test = y[features_binary['YEAR'] == test_year]

    # Initialize GaussianNB and perform Grid Search
    gnb = GaussianNB()
    gnb_grid_search = GridSearchCV(estimator=gnb, param_grid=gnb_param_grid, cv=3, n_jobs=-1, scoring='recall')
    gnb_grid_search.fit(X_train[continuous_features], y_train)
    best_gnb = gnb_grid_search.best_estimator_

    # Initialize MultinomialNB and perform Grid Search
    mnb = MultinomialNB()
    mnb_grid_search = GridSearchCV(estimator=mnb, param_grid=mnb_param_grid, cv=3, n_jobs=-1, scoring='recall')
    mnb_grid_search.fit(X_train[discrete_features], y_train)
    best_mnb = mnb_grid_search.best_estimator_

    # Get predicted probabilities for continuous and discrete features
    gnb_prob = best_gnb.predict_proba(X_test[continuous_features])[:, 1]
    mnb_prob = best_mnb.predict_proba(X_test[discrete_features])[:, 1]

    # Combine probabilities (you can average them, use weighted sum, etc.)
    combined_prob = (gnb_prob + mnb_prob) / 2

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (combined_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for GaussianNB in year {test_year}: {gnb_grid_search.best_params_}")
    print(f"Best parameters for MultinomialNB in year {test_year}: {mnb_grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]

Best parameters for GaussianNB in year 2015: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2015: {'alpha': 0.1}
Classification report for year 2015:


 10%|█         | 1/10 [00:08<01:17,  8.58s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    775271
           1       0.42      0.12      0.19      9736

    accuracy                           0.99    785007
   macro avg       0.70      0.56      0.59    785007
weighted avg       0.98      0.99      0.98    785007

Confusion Matrix for year 2015:
[[773620   1651]
 [  8539   1197]]
True Positives: 1197
False Positives: 1651
True Negatives: 773620
False Negatives: 8539
Best parameters for GaussianNB in year 2016: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2016: {'alpha': 2.0}
Classification report for year 2016:


 20%|██        | 2/10 [00:18<01:15,  9.39s/it]

...

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    797092
           1       0.06      0.04      0.05     13298

    accuracy                           0.97    810390
   macro avg       0.52      0.51      0.52    810390
weighted avg       0.97      0.97      0.97    810390

Confusion Matrix for year 2022:
[[789534   7558]
 [ 12814    484]]
True Positives: 484
False Positives: 7558
True Negatives: 789534
False Negatives: 12814
Best parameters for GaussianNB in year 2023: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2023: {'alpha': 0.1}
Classification report for year 2023:


 90%|█████████ | 9/10 [03:07<00:28, 28.48s/it]

              precision    recall  f1-score   support

           0       0.99      0.99      0.99    800711
           1       0.05      0.05      0.05     11149

    accuracy                           0.97    811860
   macro avg       0.52      0.52      0.52    811860
weighted avg       0.97      0.97      0.97    811860

Confusion Matrix for year 2023:
[[790632  10079]
 [ 10566    583]]
True Positives: 583
False Positives: 10079
True Negatives: 790632
False Negatives: 10566
Best parameters for GaussianNB in year 2024: {'var_smoothing': 1e-09}
Best parameters for MultinomialNB in year 2024: {'alpha': 0.1}
Classification report for year 2024:


100%|██████████| 10/10 [03:48<00:00, 22.86s/it]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    806672
           1       0.01      0.05      0.01      2049

    accuracy                           0.98    808721
   macro avg       0.50      0.52      0.50    808721
weighted avg       1.00      0.98      0.99    808721

Confusion Matrix for year 2024:
[[793852  12820]
 [  1944    105]]
True Positives: 105
False Positives: 12820
True Negatives: 793852
False Negatives: 1944
Year: 2015, Recall Score: 0.12294576828266228
Year: 2016, Recall Score: 0.10920018492834027
Year: 2017, Recall Score: 0.09618510373840711
Year: 2018, Recall Score: 0.029283216783216784
Year: 2019, Recall Score: 0.02676282051282051
Year: 2020, Recall Score: 0.027824151363383415
Year: 2021, Recall Score: 0.02599873176918199
Year: 2022, Recall Score: 0.036396450594074294
Year: 2023, Recall Score: 0.052291685352946454
Year: 2024, Recall Score: 0.05124450951683748
